# ONNX教程 - 第2部分：PyTorch模型创建与训练

本notebook展示如何创建、训练并保存一个简单的PyTorch模型，这是ONNX模型转换流程的第一步。我们将使用MNIST手写数字数据集训练一个卷积神经网络，并保存模型以便后续转换为ONNX格式。

本教程包含以下步骤：

1. 导入必要的库
2. 定义模型架构
3. 加载和准备数据
4. 训练模型
5. 评估模型性能
6. 保存训练好的模型

In [ ]:
# 导入必要的库
import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
# 检查是否可以使用GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")

## 1. 定义模型架构

我们将定义一个简单的卷积神经网络用于MNIST手写数字识别。模型结构如下：

- 两个卷积层，每个后面跟着ReLU激活函数和最大池化层
- 两个全连接层
- Dropout层以减少过拟合
- 输出层使用log_softmax激活函数

In [ ]:
class MNISTModel(nn.Module):
    """MNIST手写数字识别模型"""
    def __init__(self):
        super(MNISTModel, self).__init__()
        # 第一个卷积层：1通道输入，32通道输出，3x3卷积核
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        # 第二个卷积层：32通道输入，64通道输出，3x3卷积核
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        # 最大池化层
        self.pool = nn.MaxPool2d(kernel_size=2)
        # 全连接层1：将7x7x64维张量展平为1维，然后映射到128维
        self.fc1 = nn.Linear(7 * 7 * 64, 128)
        # 全连接层2：将128维映射到10维（对应10个数字类别）
        self.fc2 = nn.Linear(128, 10)
        # Dropout层，用于减少过拟合
        self.dropout = nn.Dropout(0.25)
        
    def forward(self, x):
        # 第一个卷积层+ReLU激活+最大池化
        x = self.pool(F.relu(self.conv1(x)))  # 输出尺寸: [batch, 32, 14, 14]
        # 第二个卷积层+ReLU激活+最大池化
        x = self.pool(F.relu(self.conv2(x)))  # 输出尺寸: [batch, 64, 7, 7]
        # 展平张量
        x = x.view(-1, 7 * 7 * 64)  # 输出尺寸: [batch, 7*7*64]
        # Dropout
        x = self.dropout(x)
        # 全连接层1+ReLU激活
        x = F.relu(self.fc1(x))  # 输出尺寸: [batch, 128]
        # Dropout
        x = self.dropout(x)
        # 全连接层2
        x = self.fc2(x)  # 输出尺寸: [batch, 10]
        # 输出层使用log_softmax
        return F.log_softmax(x, dim=1)

## 2. 加载和准备数据

接下来，我们将加载MNIST数据集并应用必要的预处理。

In [ ]:
def load_data():
    """加载MNIST数据集"""
    # 数据预处理和增强
    transform = transforms.Compose([
        transforms.ToTensor(),  # 将图像转换为Tensor
        transforms.Normalize((0.1307,), (0.3081,))  # 标准化（MNIST数据集的均值和标准差）
    ])
    
    # 下载并加载训练集
    train_dataset = datasets.MNIST('../data', train=True, download=True, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    
    # 下载并加载测试集
    test_dataset = datasets.MNIST('../data', train=False, transform=transform)
    test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)
    
    return train_loader, test_loader

# 加载数据
print("[1] 加载MNIST数据集")
train_loader, test_loader = load_data()
print(f"训练集大小: {len(train_loader.dataset)}")
print(f"测试集大小: {len(test_loader.dataset)}")

## 3. 创建模型实例

现在我们将创建模型实例并将其移动到适当的设备（CPU/GPU）。

In [ ]:
# 设置随机种子以便结果可复现
torch.manual_seed(42)

# 创建模型
print("[2] 创建MNIST识别模型")
model = MNISTModel().to(device)
print(model)

# 设置优化器
optimizer = optim.Adam(model.parameters(), lr=0.001)

## 4. 训练和评估函数

接下来，我们定义用于训练和评估模型的函数。

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    """训练模型的一个epoch"""
    model.train()  # 设置为训练模式
    total_loss = 0
    start_time = time.time()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        # 将数据移至GPU（如果可用）
        data, target = data.to(device), target.to(device)
        
        # 梯度清零
        optimizer.zero_grad()
        
        # 前向传播
        output = model(data)
        
        # 计算损失
        loss = F.nll_loss(output, target)
        total_loss += loss.item()
        
        # 反向传播
        loss.backward()
        
        # 更新参数
        optimizer.step()
        
        # 打印训练进度
        if (batch_idx + 1) % 100 == 0:
            print(f'训练: Epoch {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\t损失: {loss.item():.6f}')
    
    # 计算平均损失和训练时间
    avg_loss = total_loss / len(train_loader)
    elapsed = time.time() - start_time
    print(f'Epoch {epoch} 训练完成, 平均损失: {avg_loss:.6f}, 用时: {elapsed:.2f} 秒')
    
    return avg_loss

def test(model, device, test_loader):
    """评估模型性能"""
    model.eval()  # 设置为评估模式
    test_loss = 0
    correct = 0
    
    with torch.no_grad():  # 在评估时不需要计算梯度
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            # 累加批次损失
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            
            # 获取最大对数概率的索引
            pred = output.argmax(dim=1, keepdim=True)
            
            # 计算正确预测的数量
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    # 计算平均损失
    test_loss /= len(test_loader.dataset)
    
    # 打印测试结果
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'测试集: 平均损失: {test_loss:.4f}, 准确率: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)')
    
    return test_loss, accuracy

## 5. 训练模型

现在我们将训练模型并跟踪性能指标。

In [ ]:
# 训练模型
print("[3] 开始训练模型")
n_epochs = 5
train_losses = []
test_losses = []
test_accuracies = []

for epoch in range(1, n_epochs + 1):
    train_loss = train(model, device, train_loader, optimizer, epoch)
    test_loss, accuracy = test(model, device, test_loader)
    
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    test_accuracies.append(accuracy)

## 6. 保存模型

训练完成后，将模型保存到文件中，以便后续用于ONNX转换。

In [ ]:
def save_model(model, path='model.pth'):
    """保存PyTorch模型"""
    # 创建目录（如果不存在）
    os.makedirs(os.path.dirname(path) if os.path.dirname(path) else '.', exist_ok=True)
    
    # 保存模型
    torch.save(model.state_dict(), path)
    print(f'模型已保存到 {os.path.abspath(path)}')

# 保存训练好的模型
print("[4] 保存模型")
save_model(model, '../models/mnist_cnn.pth')

## 7. 显示训练结果

让我们查看训练过程中的性能指标。

In [ ]:
# 打印训练结果
print("[5] 训练结束")
print(f"最终测试准确率: {test_accuracies[-1]:.2f}%")

# 可选：绘制训练曲线
try:
    import matplotlib.pyplot as plt
    
    plt.figure(figsize=(12, 5))
    
    # 损失曲线
    plt.subplot(1, 2, 1)
    plt.plot(range(1, n_epochs + 1), train_losses, 'b-', label='训练损失')
    plt.plot(range(1, n_epochs + 1), test_losses, 'r-', label='测试损失')
    plt.xlabel('Epoch')
    plt.ylabel('损失')
    plt.legend()
    plt.title('训练和测试损失')
    
    # 准确率曲线
    plt.subplot(1, 2, 2)
    plt.plot(range(1, n_epochs + 1), test_accuracies, 'g-')
    plt.xlabel('Epoch')
    plt.ylabel('准确率 (%)')
    plt.title('测试准确率')
    
    plt.tight_layout()
    plt.show()
except ImportError:
    print("matplotlib未安装，跳过绘图")

## 8. 检查模型文件

验证模型文件是否已正确保存。

In [ ]:
# 检查保存的模型文件
model_path = '../models/mnist_cnn.pth'

if os.path.exists(model_path):
    file_size = os.path.getsize(model_path) / (1024 * 1024)  # 转换为MB
    print(f"模型文件已保存: {model_path}")
    print(f"文件大小: {file_size:.2f} MB")
    print("\n模型训练和保存完成！下一步是将PyTorch模型转换为ONNX格式（第3部分）。")
else:
    print(f"错误: 模型文件未找到: {model_path}")